## Launcher notebook

This notebook shows how to launch the flights_model.ipynb notebook either in Deep Learning VM or on Kubeflow pipelines

In [1]:
# change these to try this notebook out
BUCKET = 'hostedkfp-default-bmeiqdunw2'
PROJECT = 'ma-poc-automation'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_anaconda_depends         2019.03                  py37_0  
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                      1_llvm    conda-forge
_tflow_select             2.3.0                       mkl  
abseil-cpp                20200225.1           he1b5a44_2    conda-forge
absl-py                   0.9.0            py37hc8dfbb8_1    conda-forge
adal                      1.2.2                      py_0    conda-forge
alabaster                 0.7.12                     py_0    conda-forge
anaconda                  custom                   py37_1  
anaconda-client           1.7.2                      py_0    conda-forge
anaconda-project          0.8.3                      py_0    conda-forge
ansiwrap                  0.8.4                      py_0    conda-forge
appdirs                   1.4.3                      

### Create the pipeline

In [4]:
import kfp.components as comp
import kfp.dsl as dsl

# a single-op pipeline that runs the flights pipeline on the pod
@dsl.pipeline(
   name='FlightsPipeline',
   description='Trains, deploys flights model'
)
def flights_pipeline(
        model_name = dsl.PipelineParam('modelname'),
    model_version = dsl.PipelineParam('version'),
   inputnb=dsl.PipelineParam('inputnb'),
   outputnb=dsl.PipelineParam('outputnb'),
   params=dsl.PipelineParam('params'),
    model_bucket = dsl.PipelineParam('params')

    
):
    notebookop = dsl.ContainerOp(
      name='flightsmodel',
      # image needs to be a compile-time string
      image='gcr.io/ma-poc-automation/submitnotebook:latest',
      arguments=[
        inputnb,
        outputnb,
        params,
        model_bucket
      ],
        file_outputs={'train': '/output.txt'}
    )
    
    deploy_cmle = dsl.ContainerOp(
      name='deploycmle',
      # image needs to be a compile-time string
      image='gcr.io/ma-poc-automation/notebook-deploycmle:latest',
      arguments=[
        notebookop.outputs['train'],  # modeldir
        model_name,
        model_version
      ],
      file_outputs={
        'model': '/model.txt',
        'version': '/version.txt'
      }
    )
# compile the pipeline
pipeline_func = flights_pipeline
pipeline_filename = pipeline_func.__name__ + '.tar.gz'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)
print(pipeline_filename)

flights_pipeline.tar.gz


In [38]:
GCSDIR='gs://{}/flights/notebook'.format(BUCKET)
arguments = {
    
    'model_name' : 'flights',
    'model_version' : 'flt_1',
   'inputnb': '{}/flights_model.ipynb'.format(GCSDIR),
    'outputnb': '{}/flights_model_kfp.ipynb'.format(GCSDIR),
    'params': '{}/params.yaml'.format(GCSDIR),
    'model_bucket': BUCKET,
}


In [7]:
import kfp

import kfp
import kfp.components as comp

client = kfp.Client().create_run_from_pipeline_func(flights_pipeline, arguments=arguments)

SyntaxError: invalid syntax (experiment_service_api.py, line 127)

In [2]:
!ls *.tar.gz

flights_pipeline.tar.gz


### Run the pipeline

In [3]:
%%writefile params.yaml
BUCKET: 'hostedkfp-default-bmeiqdunw2'
PROJECT: 'ma-poc-automation'
DEVELOP_MODE: False

Overwriting params.yaml


In [5]:
%%bash
gsutil -m rm -r gs://$BUCKET/flights/notebook
gsutil cp *.ipynb params.yaml gs://$BUCKET/flights/notebook

Removing gs://hostedkfp-default-bmeiqdunw2/flights/notebook/flights_model.ipynb#1588262240510371...
Removing gs://hostedkfp-default-bmeiqdunw2/flights/notebook/launcher.ipynb#1588262240695495...
Removing gs://hostedkfp-default-bmeiqdunw2/flights/notebook/params.yaml#1588262240902064...
/ [3/3 objects] 100% Done                                                       
Operation completed over 3 objects.                                              
Copying file://flights_model.ipynb [Content-Type=application/octet-stream]...
Copying file://launcher.ipynb [Content-Type=application/octet-stream]...        
Copying file://params.yaml [Content-Type=application/octet-stream]...           
/ [3 files][ 46.9 KiB/ 46.9 KiB]                                                
Operation completed over 3 objects/46.9 KiB.                                     


In [6]:
!gsutil ls gs://$BUCKET/flights/notebook

gs://hostedkfp-default-bmeiqdunw2/flights/notebook/flights_model.ipynb
gs://hostedkfp-default-bmeiqdunw2/flights/notebook/launcher.ipynb
gs://hostedkfp-default-bmeiqdunw2/flights/notebook/params.yaml


In [7]:

GCSDIR='gs://{}/flights/notebook'.format(BUCKET)
arguments = {
    'inputnb': '{}/flights_model.ipynb'.format(GCSDIR),
    'outputnb': '{}/flights_model_kfp.ipynb'.format(GCSDIR),
    'params': '{}/params.yaml'.format(GCSDIR),
}


In [6]:

list_experiments_response = client.list_experiments()
experiments = list_experiments_response.experiments
if not experiments:
    #The user does not have any experiments available. Creating a new one
    print('here')
    experiment = client.create_experiment('Flight pipeline experiment')
else:
    experiment = experiments[-1] #Using the last experiment

#Submit a pipeline run
from datetime import datetime
run_name = 'Flight pipeline {}'.format(datetime.now().strftime("%Y%m%d %H%M%S"))
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, params=arguments)

AttributeError: type object 'ApiResourceType' has no attribute 'NAMESPACE'

In [149]:
!gsutil ls gs://$BUCKET/flights/notebook

gs://cloud-training-demos-ml/flights/notebook/flights_model.ipynb
gs://cloud-training-demos-ml/flights/notebook/flights_model_kfp.ipynb
gs://cloud-training-demos-ml/flights/notebook/flights_small.ipynb
gs://cloud-training-demos-ml/flights/notebook/launcher.ipynb
gs://cloud-training-demos-ml/flights/notebook/notebook.ipynb
gs://cloud-training-demos-ml/flights/notebook/params.yaml


In [40]:
import kfp.components as comp
import kfp.dsl as dsl

# a single-op pipeline that runs the flights pipeline on the pod
@dsl.pipeline(
   name='FlightsPipeline',
   description='Trains, deploys flights model'
)
def flights_pipeline(
        model_name = dsl.PipelineParam('modelname'),
    model_version = dsl.PipelineParam('version'),
   inputnb=dsl.PipelineParam('inputnb'),
   outputnb=dsl.PipelineParam('outputnb'),
   params=dsl.PipelineParam('params'),
    model_bucket = dsl.PipelineParam('params'),

    
):
    notebookop = dsl.ContainerOp(
      name='flightsmodel',
      # image needs to be a compile-time string
      image='gcr.io/ma-poc-automation/submitnotebook:latest',
      arguments=[
        inputnb,
        outputnb,
        params,
        model_bucket
      ],
        file_outputs={'train': '/output.txt'}
    )
    
    deploy_cmle = dsl.ContainerOp(
      name='deploycmle',
      # image needs to be a compile-time string
      image='gcr.io/ma-poc-automation/notebook-deploycmle:latest',
      arguments=[
        notebookop.outputs['train'],  # modeldir
        model_name,
        model_version
      ],
      file_outputs={
        'model': '/model.txt',
        'version': '/version.txt'
      }
    )
# compile the pipeline
pipeline_func = flights_pipeline
pipeline_filename = pipeline_func.__name__ + '.tar.gz'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)
print(pipeline_filename)

flights_pipeline.tar.gz
